# LIAR DETECTION GROUP PROJECT

Using char_wb to do character n-grams only within words.

In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
from functools import reduce

#import unittest
from IPython.display import display, HTML
#from sklearn.utils import shuffle
# NLTK for NLP utils and corpora
#import nltk
from collections import defaultdict
# NumPy and TensorFlow
import numpy as np
import pandas as pd
#import tensorflow as tf
import pickle
import dill
# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import timeit  #For timing
from sklearn.feature_extraction.text import CountVectorizer
#from project_files import pdio # for saving and loading dataframes

### Load data
Loading data from Pre-Processing step Part 1

In [2]:
#Read pkl file from part 1
# all_data = pd.read_pickle('parsed_data/df_alldata1.pkl')
all_data = pd.read_pickle('parsed_data/df_liarpolitifact_data_embed.pkl')
all_data.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23258 entries, 0 to 23257
Data columns (total 6 columns):
target            23258 non-null int64
title             23258 non-null object
title_tokcan      23258 non-null object
title_POS         23258 non-null object
binary_target     23258 non-null int64
embedded_title    23258 non-null object
dtypes: int64(2), object(4)
memory usage: 19.1 MB


In [3]:
all_data[:5]

,target,title,title_tokcan,title_POS,binary_target,embedded_title
0,1,Says 31 percent of Texas physicians accept all...,"[says, <number>, percent, of, texas, physician...","[V, $, N, P, ^, N, V, D, A, ^, N, ,, R, P, $, ...",1,"[[0.11797, 0.21126, 0.29075, -0.021211, 0.7819..."
1,2,''Both Democrats and Republicans are advocatin...,"['', both, democrats, and, republicans, are, a...","[,, D, N, &, N, V, V, P, D, N, P, N, N, V, P, ...",-1,"[[0.0028594, 0.19457, -0.19449, -0.037583, 0.9..."
2,0,A Republican-led softening of firearms trainin...,"[a, republican-led, softening, of, firearms, t...","[D, A, N, P, N, N, N, V, D, A, N, V, V, V, P, ...",1,"[[0.21705, 0.46515, -0.46757, 0.10082, 1.0135,..."
3,5,The first tweet was sent from Austin.,"[the, first, tweet, was, sent, from, austin, .]","[D, A, N, V, V, P, ^, ,]",0,"[[0.418, 0.24968, -0.41242, 0.1217, 0.34527, -..."
4,2,Georgia has the countrys second highest number...,"[georgia, has, the, countrys, second, highest,...","[^, V, D, N, A, A, N, P, A, N, N, N, ,]",-1,"[[-1.3427, 0.4592, 0.19281, 0.71305, -0.5934, ..."


# Char pre-process
Make it easier to select different sources

In [4]:
#select column to train on 
# mini_data = all_data[:10]

tok_char_n = lambda a, ngram_len: [a[i:i+ngram_len] for i in range(len(a))][:-ngram_len]

n_gram_len = 2
field_name='title'
new_field_name = field_name+'_pad'
tok_field_name = field_name+'_tok'
tok_int_field_name = field_name+'_tok_int'

# find 95% quantile
text_lens=[len(i) for i in all_data[field_name]]
pad_len = int(np.percentile(text_lens, 95))

# truncate/pad new field to 95th percentile
all_data[new_field_name] = [(" ".join(i.lower().split()) + pad_len * ' ')[:pad_len] for i in all_data[field_name]]

# tokenize
all_data[tok_field_name] = [tok_char_n(i,n_gram_len) for i in all_data[new_field_name]]

# Define helper functions
Helper functions for processing

In [5]:
# pulled from a StackOverflow post

def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test

# Process character n-grams
Create character n-grams for each of the news entries. This gives me the vocabulary.

In [6]:
# for ngram_len in range(start_ngram,start_ngram):
# vec = []
# # ngram = []
# # start_ngram = 2
# # vocab = []
# i = ngram_len-2
vec=CountVectorizer(analyzer='char_wb', binary=True, ngram_range=(n_gram_len, n_gram_len))
ngram = vec.fit_transform(all_data[field_name])
vocab = vec.vocabulary_
vocab['  '] = len(vocab)
vocab['oov'] = len(vocab)
default = len(vocab)-1

tok = lambda voc, toks: [voc.get(t,default) for t in toks]

all_data[tok_int_field_name] = [tok(vocab, j) for j in all_data[tok_field_name]]

all_data[tok_int_field_name][:10]
print(default)

1601


# Build and run Keras LSTM


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

all_data_train, all_data_dev, all_data_test = train_validate_test_split(all_data, train_percent = .7, validate_percent=.15)

x_train = np.array(list(all_data_train[tok_int_field_name]))
print("x_train: ", x_train.shape)
y_train = np.reshape(list(all_data_train['binary_target']),[-1,1])
print("y_train: ", y_train.shape)
x_test = np.array(list(all_data_dev[tok_int_field_name]))
print("x_test: ", x_test.shape)
y_test = np.reshape(list(all_data_dev['binary_target']),[-1,1])
print("y_test: ", y_test.shape)

max_features = len(vocab)

model = Sequential()
# model.add(Embedding(max_features, output_dim=256))
# model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)
score = model.evaluate(x_test, y_test, batch_size=32)


# define model
# model = Sequential()
# model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by Interac

x_train:  (16280, 190)
y_train:  (16280, 1)
x_test:  (3488, 190)
y_test:  (3488, 1)
Epoch 1/2
16280/16280 [==============================] - 1s 78us/step - loss: 13.5510 - acc: 0.3562
Epoch 2/2
3488/3488 [==============================] - 0s 42us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                12224     
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 14,337
Trainable params: 14,337
Non-trainable params: 0
_________________________________________________________________
None


list(all_data_dev[tok_int_field_name][:10])

In [8]:
# x_train = np.array(list(all_data_train[tok_int_field_name]))
# print("x_train: ", x_train.shape)
# y_train = np.array(list(all_data_train['target']))
# print("y_train: ", y_train.shape)
# x_test = np.array(list(all_data_dev[tok_int_field_name]))
# print("x_test: ", x_test.shape)
# y_test = np.reshape(list(all_data_dev['target']),[1,-1])
# print("y_test: ", y_test.shape)

score = model.evaluate(x_test, y_test, batch_size=16)

3488/3488 [==============================] - 0s 36us/step


# isot score

In [9]:
score

[13.565653651132497, 0.3540711009174312]

In [10]:
default

1601

In [13]:
from IntegratedGradients.IntegratedGradients import *

ig = integrated_gradients(model)
ig.explain(x_test[256])

Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])